In [2]:
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

In [3]:
# Connect to a server.
connections.connect("default", host="localhost", port="19530")  # 0.0.0.0:19530

In [4]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="random", dtype=DataType.DOUBLE),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8)
]
schema = CollectionSchema(fields, "hello_milvus is the simplest demo to introduce the APIs")
hello_milvus = Collection("hello_milvus", schema)

In [5]:
import random
entities = [
    [i for i in range(3000)],  # field pk
    [float(random.randrange(-20, -10)) for _ in range(3000)],  # field random
    [[random.random() for _ in range(8)] for _ in range(3000)],  # field embeddings
]
insert_result = hello_milvus.insert(entities)
# After final entity is inserted, it is best to call flush to have no growing segments left in memory
hello_milvus.flush()  # Need to flush after inserts, before indexing.

In [6]:
# Build indexes on entities.
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
hello_milvus.create_index("embeddings", index)

Status(code=0, message=)

In [7]:
# Load the collection to memory and perform vector similarity search.
hello_milvus.load()
vectors_to_search = entities[-1][-2:]
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, output_fields=["random"])

In [8]:
result

In [9]:
# Perform a vector query.
result = hello_milvus.query(expr="random > -14", output_fields=["random", "embeddings"])

In [10]:
result

[{'random': -12.0,
  'embeddings': [0.03677968,
   0.7891713,
   0.06463133,
   0.104057714,
   0.52169317,
   0.18915801,
   0.47587034,
   0.41397172],
  'pk': 0},
 {'random': -13.0,
  'embeddings': [0.835029,
   0.16265528,
   0.69179654,
   0.2151936,
   0.83096534,
   0.17723505,
   0.69857645,
   0.6024629],
  'pk': 4},
 {'random': -11.0,
  'embeddings': [0.68885094,
   0.17241697,
   0.5057628,
   0.5143066,
   0.17879719,
   0.95259243,
   0.6712088,
   0.7405284],
  'pk': 5},
 {'random': -12.0,
  'embeddings': [0.36389306,
   0.8279691,
   0.52653116,
   0.021390246,
   0.2963606,
   0.6841746,
   0.43795606,
   0.8700084],
  'pk': 6},
 {'random': -12.0,
  'embeddings': [0.107242316,
   0.23648979,
   0.7916818,
   0.22534932,
   0.052980375,
   0.49898008,
   0.9541613,
   0.8472938],
  'pk': 7},
 {'random': -11.0,
  'embeddings': [0.8253513,
   0.62312245,
   0.568482,
   0.724228,
   0.33627293,
   0.89384145,
   0.6595406,
   0.15621382],
  'pk': 8},
 {'random': -13.0,
  '

In [11]:
# Perform a hybrid search.
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, expr="random > -12", output_fields=["random"])

In [12]:
result

In [13]:
# !TODO
# Delete entities by their PKs. CHECK THIS!
expr = f"pk in [{entities[0]}, {entities[1]}]"
hello_milvus.delete(expr)

RPC error: [delete], <MilvusException: (code=1, message=failed to create expr plan, expr = pk in [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 2

MilvusException: <MilvusException: (code=1, message=failed to create expr plan, expr = pk in [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 

In [14]:
# Drop a collection.
utility.drop_collection("hello_milvus")